## A. Configuración General.

In [ ]:
#1. Librerías.
%run "../librerias.ipynb"

c:\Users\c678456\AppData\Local\anaconda3\envs\dmeyf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#2. Constantes.
#a. Constantes generales.
%run "../constantes.ipynb"

#b. Constantes particulares a modificar por el usuario.
dataset_con_fe = dataset_file_fe_12_3_DQ

cantidad_meses_train = 12
ventana = 3

mes_train = mes_train_ult_anio
mes_test = mes_test

# Ruta de la BBDD donde se almacenan los hiperparámetros óptimos post-Optuna.
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = f"exp_lgbm_{cantidad_meses_train}_{ventana}_undersampling_DQ_B2"

In [ ]:
#3. Funciones
%run "../funciones.ipynb"

In [ ]:
#4. Lectura de datos.
data = pd.read_parquet(dataset_con_fe)

In [ ]:
#5. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)

#ii. Pesos y reclusterización.
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data['clase_binaria2'] = 0
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

#iii. Divido entre Train y Test.
train_data = data[data['foto_mes'].isin(mes_train)]
test_data = data[data['foto_mes'] == mes_test]

#iv. Separar en X e y.
#a. Datos para entrenar todo el modelo final para Kaggle.
X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

#b. Datos de Test (a predecir).
X_test = test_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)

In [ ]:
#6. Voy a leer el estudio de Optuna,
study = optuna.load_study(
    study_name=study_name,
    storage=storage_name
    )

## B. Entreno el modelo.

In [ ]:
#1. Tomamos el mejor modelo y con eso entrenamos todos los datos.
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[0],
    'verbose': 0
}

train_data_modelo = lgb.Dataset(X_train,
                          label=y_train_binaria2,
                          weight=w_train)

model_lgb = lgb.train(params,
                  train_data_modelo,
                  num_boost_round=best_iter)

## C. Guardamos el modelo entrenado.

In [ ]:
#1. Guardamos el modelo.
# Primer dígito cuantos meses para atrás desde 06/21, segundo dígito número data drifting, tercer dígito número de entrenamiento.
model_lgb.save_model(modelos_path + 'lgbm_{}_{}_undersampling_DQ_B2.txt'.format(cantidad_meses_train,ventana))